In [ ]:
# -*- coding:utf-8 -*-


# general imports
import os.path as op
import os

# third party imports
import numpy as np
import nibabel as nib
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt

require_labels = [0,    # background
                  1,    # spleen
                  2,    # right kidney
                  3,    # left kidney
                  6]    # liver

resample_size = [256,256,32]
image_path = '../../Dataset/BTCV/data/'
save_path = '../../Dataset/BTCV/BTCV_%d_%d_%d/' % \
            (resample_size[0], resample_size[1], resample_size[2])
process_list=['train', 'test']

In [3]:
def read_nii( volume_path):
    nii = nib.load(volume_path)
    data = nii.get_fdata()
    header = nii.header
    affine = nii.affine
    spacing = list(nii.header['pixdim'][1:4])
    return data, header, affine, spacing

def create_path(pathlist):
    for path in pathlist:
        if not op.exists(path):
            os.makedirs(path)
            
def save_nii( data, save_path, header, affine):
    new_img = nib.Nifti1Image(data, affine, header)
    nib.save(new_img, save_path)

def resize(img, label, spacing, resample_size):
    assert img.shape == label.shape
    origin_shape = img.shape
    new_space = spacing * origin_shape / resample_size
    resize_factor = 1.0 * np.asarray(resample_size) / np.asarray(origin_shape)
    print('resize_factor: ', resize_factor)
    img = zoom(img, resize_factor, order=1) #order = 1：bilinear interpolaion
    label = zoom(label, resize_factor, order=0) #order = 0:nearest
    return img, label, new_space
         
def create_and_save_resize_data(resample_size, img, label, patient,
                                spacing,img_header, img_affine,save_path):
    nii_save_path = op.join(save_path, str(patient-43))
    create_path([nii_save_path])
    img, label, new_space = resize(img, label, np.array(spacing), resample_size)
    label = np.round(label)
    img_header['pixdim'][1:4] = new_space
    save_nii(img, op.join(nii_save_path, 'procimg.nii.gz'), img_header, img_affine)
    save_nii(label, op.join(nii_save_path, 'seg.nii.gz'), img_header, img_affine)
    return img, label

In [ ]:
for patient in range(44,91): 
    
    print("Proprecess train image " + str(patient))
    img, img_header, img_affine, img_spacing = read_nii(
        op.join(image_path, 'image_'+str(patient)+'.nii.gz'))
    label, label_header, label_affine, label_spacing = read_nii(
        op.join(image_path, 'label_'+str(patient)+'.nii.gz'))
    print('Image and label shape before resize: ', img.shape, label.shape)
    print(img_spacing)
    print(label.dtype)
    #affine:voxel position to position in reference space

    label_unique = np.unique(label)
    label_unique = np.delete(label_unique, np.where(label_unique == 0))
    print(label_unique)
    print(len(label_unique))
    for l in label_unique:
        if l not in require_labels:
            label[label == l] = 0 

    if patient-43 in (6, 7, 9, 12, 15, 17, 21, 22, 28, 30): # Random selection validation set
        save_path_ = save_path + '/valid/'
    else :
        save_path_ = save_path + '/train/'
    img, label = create_and_save_resize_data(resample_size, img, label,
                                                  patient, img_spacing,img_header, 
                                                  img_affine, save_path_)
    
    print('Image and label shape after resize: ', img.shape, label.shape)
    fig,axes = plt.subplots(1,2)
    ax1 = axes[0]
    ax2 = axes[1]

    ax1.imshow(img[:,:,20])
    ax2.imshow(label[:,:,20])
    plt.show()
    print()